# DI Pre Processing
Corrects png files and store them in a new folder
<br>
<br>
Usage: 
1) Run notebook
2) Enter drive where video files are stored 
<br>
<br>

Notes:
1) Needs JSON files that contains metadata
2) Update root directory
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [19]:
#import libraries
import os
import cv2
import json
import pandas as pd
import numpy as np
from timeit import default_timer as timer
import psutil
import traceback

In [20]:
# make sure to update path
user_drive = input("Enter user drive: ").upper()
video_path = f"{user_drive}:/Christian/DI_centre_structured"
input(f"Is this the right directory - {video_path}?")

''

In [21]:
VIDEO_CHARACTERISTICS = {
    "With Blankets" : "WB",
    "B" : "WB",
    "WB": "WB",
    "Without Blankets" : "WOB",
    "WOB": "WOB",
    "3 Meters" : "3m",
    "2 Meters" : "2m",
    "Hold Breath" : "HB",
    "Hold Breathe" : "HB",
    "H" : "HB",
    "Relaxed" : "rel",
    "R": "rel",
}

In [22]:
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"
log_dir = repo_dir + "/records/logs"

In [23]:
""" creates unique id for each scenario """

def get_id(patient_data: dict, video_count:int) -> str:
    alias = patient_data["alias"]
    blanket = VIDEO_CHARACTERISTICS.get(patient_data["blanket"], "?")
    distance = VIDEO_CHARACTERISTICS.get(patient_data["distance"].title(), "?")
    breathing = VIDEO_CHARACTERISTICS.get(patient_data["breathing"], "?")
    id = alias + "-" + distance + "-" + blanket + "-" + breathing
    id = f"{alias}_{video_count}-{distance}-{blanket}-{breathing}"
    return id

In [24]:
""" creates folder for exported frames if it does not exist """

def set_folder(save_folder:str) -> None:
    if not os.path.isdir(save_folder):
        os.mkdir(save_folder)
    else:
        for files in os.listdir(save_folder):
            os.remove(os.path.join(save_folder, files))

In [25]:
class NotEnoughFreeSpace(Exception):
    pass

In [26]:
""" gets drives storage """

def get_drive_storage(user_drive:str):
    try:
        drive_usage = psutil.disk_usage(user_drive + ":\\")
        
        # Convert bytes to GB
        total_space = drive_usage.total / (1024**3)
        used_space = drive_usage.used / (1024**3)
        free_space = drive_usage.free / (1024**3)
        
        return {
            'drive_letter': user_drive,
            'total_space_GB': total_space,
            'used_space_GB': used_space,
            'free_space_GB': free_space,
        }
    except FileNotFoundError:
        return f"Drive {user_drive} not found."

def check_drive_usage(user_drive:str):
    minimum_space = 75
    drive_info = get_drive_storage(user_drive)

    if drive_info["free_space_GB"] < minimum_space:
        raise NotEnoughFreeSpace(f"Not enough free space on Drive {user_drive}. Minimum {minimum_space} GB required.")    

In [27]:
""" re samples number of frames """

def up_sample(old_fps: int, new_fps: int, start:int) -> list[int]:
    frames_arr = np.arange(start, old_fps, dtype=int)
    frames_interp = np.linspace(start, old_fps-1, new_fps)
    nearest_indices = np.round(frames_interp).astype(int)
    up_sampled_list = np.take(frames_arr, nearest_indices, mode='wrap')

    return up_sampled_list.tolist()

In [28]:
""" checks each png for corruption """

def is_png_corrupted(file_path):
    try:
        image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
        if image is None:
            return True
        return False
    except Exception:
        return True

def check_for_corrupted_png_files(folder_path):
    corrupted_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".png"):
                file_path = os.path.join(root, file)
                if is_png_corrupted(file_path):
                    corrupted_files.append(file_path)
    
    if len(corrupted_files) != 0:
        print("Corrupted PNG files found:")
        for file_path in corrupted_files:
            print(file_path)           
    
    return corrupted_files

In [29]:
""" gets folder paths """
def get_frames_path(local_path: str, level:str) -> list["str"]:
    fixed_path = local_path.replace("\\", "/")
    fixed_path_split = fixed_path.split("/")
    video_folder = "/".join(fixed_path_split[:-1])
    video_filename = fixed_path_split[-1].split(".")[0]
    folder_path = video_folder + f"/frames_{video_filename}_{level}"
    return [folder_path, video_folder]

In [ ]:
def run_correct_png(all_patients:dict, level: str, new_fps:int, user_drive:str) -> list[str]:
    pass

In [30]:
""" driver code for re sampling png files """
def correct_png(all_patients:list, level: str, new_fps:int, user_drive:str) -> list[str]:
    visited_folders = {}  
    video_folder_list = []

    for patient_idx, patient_info in enumerate(all_patients):
        try:
            video_path = patient_info["local path"]
            old_fps = int(patient_info["old fps"])
            
            frames_folder, video_folder = get_frames_path(video_path, level)
            video_folder_list.append(frames_folder)
            visited_folders[video_folder] = visited_folders.get(video_folder, 0) + 1
            video_count = visited_folders[video_folder]

            frames_to_pick = up_sample(old_fps, new_fps, 1)
            frames_idx = pd.Index(frames_to_pick, name="frames")
            frame_frequency = frames_idx.value_counts()

            if len(frames_to_pick) != new_fps:
                raise ValueError("Number of frames to pick is not equal to new fps")

            # get id
            patient_id = get_id(patient_info, video_count)

            print(video_path, frames_folder, new_fps, patient_id)
            print(frame_frequency)
            
        except Exception as e:
            traceback.print_exc()
            print(f'''{type(e)}: {e} for video {patient_info["filename"]}''')
        
        break
    
    return video_folder_list

In [31]:
""" reads JSON file """
def load_json(json_dir:str, filename:str) -> list[dict]:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

In [32]:
def save_folder(list_of_paths:list, log_filename:str) -> None:
    if os.path.exists(log_filename):
        os.remove(log_filename)
    
    with open(log_filename, "a") as log_data:
        for folder in list_of_paths:
            log_data.write(f"{folder}\n")

## Video to frames (stage 4.3)
Based on the desired fps, pick certain frames

In [33]:
""" local vars"""

thermal_fps = {
    "lower_bound": 5,
    "upper_bound": 10
}

In [34]:
""" load JSON files """

metadata_thermal = load_json(json_dir, "/thermal_png_complete.json")

In [35]:
""" log files to store exported frames """

thermal_logfile_prefix = log_dir + "/exported/thermal-exported"

In [36]:
""" corrects png frames """

for level, new_fps in thermal_fps.items():
    print(f"\nAdjusting FPS to {new_fps}\n" + "="*50)
    thermal_logfile = thermal_logfile_prefix + "-" + level + "-2.log"
    visited_folders = correct_png(metadata_thermal, level, new_fps, user_drive)


Adjusting FPS to 5
E:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_1/2 meters/WB/H/frames E:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_1/2 meters/WB/H/frames_frames_lower_bound 5 15_1_1-2m-WB-HB
frames
2    1
3    1
5    1
6    1
1    1
Name: count, dtype: int64

Adjusting FPS to 10
E:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_1/2 meters/WB/H/frames E:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_1/2 meters/WB/H/frames_frames_upper_bound 10 15_1_1-2m-WB-HB
frames
3    2
4    2
5    2
6    2
2    1
1    1
Name: count, dtype: int64
